In [30]:
import re
import spacy
nlp = spacy.load("en_core_web_sm")
stemmer = PorterStemmer()

def preprocess_synopsis(text):
    text.replace("\n"," ")
    text.replace("\r","")
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower()).replace('\n', '').replace('\r', '')
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

import pandas as pd

community_stB_train = pd.read_csv(".\OnlyText\stB\community_stB_train.csv")
individual_stB_train = pd.read_csv(".\OnlyText\stB\individual_stB_train.csv")
org_stB_train = pd.read_csv(".\OnlyText\stB\org_stB_train.csv")
stB_eval = pd.read_csv(".\OnlyText\stB\stB_eval.csv")
stB_test = pd.read_csv(".\OnlyText\stB\stB_test.csv")
eval_subtaskB = pd.read_csv(".\OnlyText\stB\eval_subtaskB.csv")

def labeler(sample_id, df):
    for column, row in df.iterrows():
        if str(row['index']) == str(sample_id):
            a = int(row['Label'])
            if a == 0:
                return 0 #'individual'
            elif a == 1:
                return 1 #'community'
            elif a == 2:
                return 2 # 'organization'
    print("Function failed")
    
training_data = pd.DataFrame(columns=['Sample', 'Text', 'Label'])
ind = 0 # 'individual' #0
com = 1 # 'community' #1
org = 2 # 'organization' #2
count = 1
for column, row in stB_eval.iterrows():
    sample_id = row['filename'].replace("/content/drive/MyDrive/CASE2023_Task4/CASE2023_TASK4_EvalData/subtaskB/","").replace(".jpg","")
    sample_text = preprocess_synopsis(row['text'])
    training_data.loc[count] = [sample_id, sample_text, labeler(sample_id, eval_subtaskB)]
    count+=1
for column, row in community_stB_train.iterrows():
    sample_id = row['filename'].replace("/content/drive/MyDrive/CASE2023_Task4/CASE2023_TASK4_TrainData/subTaskB/Community/","").replace(".jpg","")
    sample_text = preprocess_synopsis(row['text'])
    training_data.loc[count] = [sample_id, sample_text, com]
    count+=1
for column, row in individual_stB_train.iterrows():
    sample_id = row['filename'].replace("/content/drive/MyDrive/CASE2023_Task4/CASE2023_TASK4_TrainData/subTaskB/Individual/","").replace(".jpg","")
    sample_text = preprocess_synopsis(row['text'])
    training_data.loc[count] = [sample_id, sample_text, ind]
    count+=1
for column, row in org_stB_train.iterrows():
    sample_id = row['filename'].replace("/content/drive/MyDrive/CASE2023_Task4/CASE2023_TASK4_TrainData/subTaskB/Organization/","").replace(".jpg","")
    sample_text = preprocess_synopsis(row['text'])
    training_data.loc[count] = [sample_id, sample_text, org]
    count+=1
    
print(training_data)

     Sample                                               Text  Label
1     10757  breaking news american democrat blames putins ...      2
2     10832  nt stand ukrainians brave resistance putinist ...      2
3     11097  ship catastrophic accident sank response crews...      0
4     11140  fox news channel tuckers thoughts russias acti...      0
5     11967  nt know going happen pretty good idea putin go...      0
...     ...                                                ...    ...
2182  99524               live farming simulator   starecatcom      2
2183  99587  loses small city ukraine k took early loses   ...      2
2184  99679                 kyiv towing collect tank big small      2
2185  99712  vere hems   valin saman san tit kyiv makes nig...      2
2186  99727  donbass civilians constantly attacked ukraine ...      2

[2186 rows x 3 columns]


In [31]:
test_datas = pd.DataFrame(columns=['Sample', 'Text'])
count = 1
for column, row in stB_test.iterrows():
    sample_id = row['filename'].replace("/content/drive/MyDrive/CASE2023_Task4/CASE2023_TASK4_TestData/subtaskB/","").replace(".jpg","")
    sample_text = row['text']
    test_datas.loc[count] = [sample_id, sample_text]
    count+=1
    
print(test_datas)

    Sample                                               Text
1    10125  www.bh Future Fact! @MACFacts Genius When Pout...
2    10525  THE STANLEY KUBRICK'S SHINING HEEERE'S VLADDY ...
3    11286  AND THEN ZELENSKY SAID... WE'VE GOT A FAKE AIR...
4    11436  Russian troops firing at Ukrainian civilians R...
5    12139  2 wars are happening It only is Russia, Ukrain...
..     ...                                                ...
238  97985             RUSSIA Me trying to live a normal life
239  98310  WE SHOULD LOOK FOR THE WAYS TO SAVE PUTIN'S FA...
240  99145  WE WILL NOT ESTABLISH A 'NO FLY ZONE' IN UKRAI...
241  99821  I TRIED WARNING Y'ALL ABOUT TRUMP AND PUTIN'S ...
242  99933  Just blame everything on Russia. imgflip.com Y...

[242 rows x 2 columns]


In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score
import torch

# Step 1: Prepare the environment
df = training_data

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Step 2: Tokenize the text
def tokenize_text(text):
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        return_attention_mask=True,
        truncation=True,
        return_tensors='pt'
    )
    return encoded['input_ids'], encoded['attention_mask']

train_tokens = train_df['Text'].apply(tokenize_text)
test_tokens = test_df['Text'].apply(tokenize_text)

# Step 3: Prepare the data for BERT
train_inputs = torch.cat([token[0] for token in train_tokens])
train_masks = torch.cat([token[1] for token in train_tokens])
train_labels = torch.tensor(train_df['Label'].values)

test_inputs = torch.cat([token[0] for token in test_tokens])
test_masks = torch.cat([token[1] for token in test_tokens])
test_labels = torch.tensor(test_df['Label'].values)

# Step 4: Fine-tune BERT for classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

optimizer = AdamW(model.parameters(), lr=2e-5)

batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_loader = DataLoader(train_data, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_loader = DataLoader(test_data, batch_size=batch_size)

epochs = 4

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]

        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)

    # Evaluation on the test set
    model.eval()
    total_correct = 0
    true_labels = []
    predicted_labels = []

    for batch in test_loader:
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        _, predicted = torch.max(logits, dim=1)
        total_correct += (predicted == labels).sum().item()

        true_labels.extend(labels.numpy())
        predicted_labels.extend(predicted.numpy())

    accuracy = total_correct / len(test_df)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')

    print(f'Epoch {epoch + 1}: Train Loss = {avg_train_loss}, Test Accuracy = {accuracy}, F1 Score = {f1}')

#Epoch 1: Train Loss = 0.9558789600025523, Test Accuracy = 0.634703196347032, F1 Score = 0.566007690153761
#Epoch 2: Train Loss = 0.7432595472444188, Test Accuracy = 0.6506849315068494, F1 Score = 0.6126640848053292
#Epoch 3: Train Loss = 0.49651158079504965, Test Accuracy = 0.6529680365296804, F1 Score = 0.639967891094413
#Epoch 4: Train Loss = 0.2876184206117283, Test Accuracy = 0.634703196347032, F1 Score = 0.6193736367093801

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch 1: Train Loss = 0.9546893810684031, Test Accuracy = 0.6210045662100456, F1 Score = 0.5532251713353605
Epoch 2: Train Loss = 0.7840688060630452, Test Accuracy = 0.6757990867579908, F1 Score = 0.6555492660552441
Epoch 3: Train Loss = 0.6123944539915431, Test Accuracy = 0.6872146118721462, F1 Score = 0.6730711116068794
Epoch 4: Train Loss = 0.46432061073454944, Test Accuracy = 0.6461187214611872, F1 Score = 0.6235876474949469


In [33]:
def predict_text(text):
    inputs = tokenize_text(text)
    input_ids = inputs[0]
    attention_mask = inputs[1]

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]

    _, predicted_label = torch.max(logits, dim=1)
    return predicted_label.item()

predicciones_chidas = []
for column, row in test_datas.iterrows():
    sample = row['Sample']
    texto = row['Text']
    predicted_label = predict_text(texto)
    linea = '{"index": '+ str(sample)+ ', "prediction": '+ str(predicted_label)+ '}'
    predicciones_chidas.append(linea)

In [34]:
import json

jason = open('BERT_submissionSTB_prepros.json', 'w')
for i in range(len(predicciones_chidas)-1):
    jason.write(predicciones_chidas[i])
    jason.write('\n')
jason.write(predicciones_chidas[-1])
jason.close()